In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from datasets import load_dataset,concatenate_datasets

In [2]:
wiki_ds = load_dataset("csv", data_files="/kaggle/input/wiki-cc-openweb/wiki_small_50k/wiki_small_50k.csv")
owt_ds = load_dataset("csv", data_files="/kaggle/input/wiki-cc-openweb/openweb_50k (2)/openweb_50k.csv")
cc_news_ds = load_dataset("csv", data_files="/kaggle/input/wiki-cc-openweb/cc_news/cc_news.csv")
train_ds = concatenate_datasets([wiki_ds["train"],owt_ds["train"],cc_news_ds["train"]])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
train_ds

Dataset({
    features: ['text'],
    num_rows: 170824
})

In [131]:
from tokenizers import ByteLevelBPETokenizer
special_tokens = ['<s>','<pad>','</s>','<unk>','<mask>']
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator([x['text'] for x in train_ds], vocab_size = 10_000,min_frequency =2, special_tokens=special_tokens)


In [132]:
tokenizer.save_model("/kaggle/working/")

['/kaggle/working/vocab.json', '/kaggle/working/merges.txt']

In [133]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("/kaggle/working/")

In [64]:
train_ds['text'][1]

"Cheryl S. McWatters Cheryl S. McWatters is professor and Father Edgar Thivierge Chair in Business History at the Telfer School of Management, University of Ottawa.\n\nEducation and career\nShe was previously a professor at the University of Alberta and associate professor at McGill University. McWatters is a qualified accountant and earned her B.A., M.B.A. and Ph.D. all from Queen's University, Kingston, Ontario. Her work relates primarily to seventeenth and eighteenth century international trading networks. She is a trustee and former president of the Academy of Accounting Historians. McWatters established the Geraldine Grace and Maurice Alvin McWatters Visiting Fellowship at Queen's University in memory of her parents.\n\nEditing\nMcWatters is editor of Accounting History Review and associate editor of the Journal of Operations Management and Accounting Perspectives. She serves on the editorial boards of Accounting, Auditing and Accountability Journal and Accounting Historians Journ

In [134]:
#Sample Text Data

#from transformers import RobertaTokenizer
#tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

texts = [
   "Cheryl S. McWatters Cheryl S. McWatters is professor and Father Edgar Thivierge Chair in Business History at the Telfer School of Management, University of Ottawa.\n\nEducation and career\nShe was previously a professor at the University of Alberta and associate professor at McGill University. McWatters is a qualified accountant and earned her B.A., M.B.A. and Ph.D. all from Queen's University, Kingston, Ontario. Her work relates primarily to seventeenth and eighteenth century international trading networks. She is a trustee and former president of the Academy of Accounting Historians. McWatters established the Geraldine Grace and Maurice Alvin McWatters Visiting Fellowship at Queen's University in memory of her parents.\n\nEditing\nMcWatters is editor of Accounting History Review and associate editor of the Journal of Operations Management and Accounting Perspectives. She serves on the editorial boards of Accounting, Auditing and Accountability Journal and Accounting Historians Journal.\n\nAwards\nShingo Research Prize for Excellence in Manufacturing Research.\n\nSelected publications\nHistoire des entreprises du transport: Évolutions comptables et managériales. Paris, France: LHarmattan ''",
    "This is a small test.",
    "Roberta can learn from  examples.",
    "Short text.",
    "A slightly longer sentence for testing."
]

max_length = 32  # truncate any sequence longer than this

# Tokenize each text individually with truncation
train_dataset = []
for text in texts:
    enc = tokenizer(text, truncation=True, max_length=254,padding="max_length", return_tensors="pt")
    train_dataset.append({
        "input_ids": enc["input_ids"].squeeze(0),
        "attention_mask": enc["attention_mask"].squeeze(0)
    })

In [141]:
# Real text
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
train_dataset = []
for text in train_ds['text'][:5]:
    enc = tokenizer(text, truncation=True, max_length=254, return_tensors="pt")
    train_dataset.append({
        "input_ids": enc["input_ids"].squeeze(0),
        "attention_mask": enc["attention_mask"].squeeze(0)
    })


In [139]:
train_dataset[0]['input_ids']

tensor([    0, 33531,   735,  3138,  2897,  7393,   735,  3138,    36,   844,
          392,   504,  4156,   126,   316,   644, 34094,    43,    21,    41,
         2370,  3551,   661,     4, 50118, 50118, 16431,   254, 50118,   894,
         8069,    13,    65,    76,    19, 15437,   139, 13513, 13528,     6,
           13,  2661,    37,  8273,  1182,  1720,    36, 16883,    39,   317,
           11, 12858,    77,   373,  2115,   238,     8,    21,    39,  5548,
        25209,     4,    91,   439,    15,     7,  6396,    23,     5,  2930,
         3536,     9,  3920,     6,  2930,  1821,     9,  3920,     8,  1745,
           18,  1821,     6,  6912,   131,    39,   521,  1165, 16876,  2575,
         8876,     8, 10841, 15474,  7287,     6,    54,   258,  1059,  3615,
         3059,    19,  7393,  1448,  6276,     4,    91,   702,    19, 24896,
          661,  3351,  3889,  1488,   757,     6,     8,  4829,    20,   928,
         6892,   139,    19, 24896,   661, 23790,  4061,  4616, 

In [7]:
from transformers import (
    RobertaConfig,
    RobertaForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)

2025-08-22 13:30:42.717891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755869443.106792      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755869443.199402      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [135]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


In [39]:
#for i in range(15):
    #print(data_collator([train_dataset[i]]))

In [9]:
config = RobertaConfig(
vocab_size = tokenizer.vocab_size,
hidden_size = 256, #Embedding size
num_hidden_layers = 6,#Number of encoder layers
num_attention_heads=4, # Attentions per encoder layer, divisible by 256
intermediate_size = 1024, #FFN layer size
max_position_embeddings=256, #Position Embedding max size = embedding size
type_vocab_size =1,
layer_norm_eps = 1e-5,
initializer_range = 0.02,
pad_token_id = 1,
bos_token_id = 0,
eos_token_id = 2
)
model = RobertaForMaskedLM(config)


In [10]:
tokenizer.vocab_size

50265

In [140]:
# 4️⃣ Training arguments
training_args = TrainingArguments(
    output_dir="./checkpoints",
    do_train=True,
    save_steps=1,
    logging_steps=1,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    fp16=False,
    report_to=[]
)

# 5️⃣ Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)
trainer.train()

Step,Training Loss
1,10.677200
2,10.795900
3,10.715800


TrainOutput(global_step=3, training_loss=10.729628880818685, metrics={'train_runtime': 6.6352, 'train_samples_per_second': 0.754, 'train_steps_per_second': 0.452, 'total_flos': 36999984420.0, 'train_loss': 10.729628880818685, 'epoch': 1.0})

Step,Training Loss


IndexError: index out of range in self

In [ ]:
config = RobertaConfig(

vocab_size = tokenizer.vocab_size,

hidden_size = 256, #Embedding size

num_hidden_layers = 6,#Number of encoder layers

num_attention_heads=4, # Attentions per encoder layer, divisible by 256

intermediate_size = 1024, #FFN layer size

max_position_embeddings=256, #Position Embedding max size = embedding size

type_vocab_size =1,

layer_norm_eps = 1e-5,

initializer_range = 0.02,

pad_token_id = 1,

bos_token_id = 0,

eos_token_id = 2

)

model = RobertaForMaskedLM(config)

#model.resize_token_embeddings(len(tokenizer))

training_args = TrainingArguments(

output_dir = "./checkpoints/",
do_train = True,

overwrite_output_dir = False,

num_train_epochs=1,

per_device_train_batch_size =16,

gradient_accumulation_steps=2,

learning_rate = 5e-4,

weight_decay = 0.01,

warmup_steps = 100,

logging_dir = "./logs/",

logging_steps = 1,

save_strategy="steps",

save_steps =1,

save_total_limit=3,

fp16= True,

dataloader_num_workers=2,

)

In [ ]:
from transformers import logging
logging.set_verbosity_info()

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=data_collator,
)

In [ ]:

try:
    trainer.train()
except Exception as e:
    print("Trainer error:", e)

In [ ]:
import os

os.listdir("/kaggle/working/checkpoints")

In [ ]:
import torch
print(torch.cuda.is_available(), torch.cuda.device_count())

In [ ]:
print(len(tokenized_ds))

In [ ]:
for i, batch in enumerate(tokenized_ds):
    print(i, batch)
    if i > 5: break

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch

training_args = TrainingArguments(
    output_dir="./checkpoints",
    do_train=True,
    save_steps=1,
    logging_steps=1,
    num_train_epochs=1,
    per_device_train_batch_size=2,
)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Tiny dataset
train_dataset = [{"input_ids": torch.tensor([101, 102, 103]), "attention_mask": torch.tensor([1,1,1]), "labels": torch.tensor(1)}] * 10


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, Trainer, TrainingArguments
import torch

# tokenizer & model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForMaskedLM.from_pretrained("roberta-base")

# texts
texts = [
    "Hello, I am a [MASK] model.",
    "This is a small test [MASK].",
    "Roberta can learn from [MASK] examples."
] * 5

# batch tokenize & pad
encodings = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# build dataset
train_dataset = [
    {
        "input_ids": encodings["input_ids"][i],
        "attention_mask": encodings["attention_mask"][i],
        "labels": encodings["input_ids"][i]
    }
    for i in range(len(texts))
]

# training args
training_args = TrainingArguments(
    output_dir="./checkpoints",
    do_train=True,
    save_steps=1,
    logging_steps=1,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    fp16=False,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()


In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# 1️⃣ Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForMaskedLM.from_pretrained("roberta-base")

# 2️⃣ Tiny toy dataset (variable-length sentences)
texts = [
    "Hello, I am a [MASK] model.",
    "This is a small test [MASK].",
    "Roberta can learn from [MASK] examples.",
    "Short text.",
    "A slightly longer sentence for testing."
]

# Tokenize each text individually without padding
train_dataset = []
for text in texts:
    enc = tokenizer(text, truncation=True, return_tensors="pt")
    train_dataset.append({
        "input_ids": enc["input_ids"].squeeze(0),
        "attention_mask": enc["attention_mask"].squeeze(0)
    })

# 3️⃣ Data collator for MLM
# Automatically pads each batch and creates masked labels
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# 4️⃣ Training arguments
training_args = TrainingArguments(
    output_dir="./checkpoints",
    do_train=True,
    save_steps=1,           # save every step
    logging_steps=1,        # log every step
    num_train_epochs=1,
    per_device_train_batch_size=2,
    fp16=False,             # safer for Kaggle
    report_to=[]            # disable W&B
)

# 5️⃣ Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

# 6️⃣ Run training
trainer.train()


In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# 1️⃣ Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForMaskedLM.from_pretrained("roberta-base")

# 2️⃣ Tiny toy dataset (variable-length sentences)
texts = [
    "Hello, I am a [MASK] model.",
    "This is a small test [MASK].",
    "Roberta can learn from [MASK] examples.",
    "Short text.",
    "A slightly longer sentence for testing."
]

max_length = 32  # truncate any sequence longer than this

# Tokenize each text individually with truncation
train_dataset = []
for text in texts:
    enc = tokenizer(text, truncation=True, max_length=max_length, return_tensors="pt")
    train_dataset.append({
        "input_ids": enc["input_ids"].squeeze(0),
        "attention_mask": enc["attention_mask"].squeeze(0)
    })

# 3️⃣ Data collator for MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# 4️⃣ Training arguments
training_args = TrainingArguments(
    output_dir="./checkpoints",
    do_train=True,
    save_steps=1,
    logging_steps=1,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    fp16=False,
    report_to=[]
)

# 5️⃣ Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

# 6️⃣ Run training
trainer.train()
